In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from scipy.stats.qmc import LatinHypercube

import os
os.environ['KERAS_BACKEND'] = 'torch'
import torch
import keras
from keras import layers

In [ ]:
Nbd = 100
N = 5000

xt_bd = np.vstack((
    np.vstack((np.linspace(-1,1,Nbd),np.zeros(Nbd))).transpose(),
    np.vstack((-np.ones(Nbd),np.linspace(1/Nbd,1,Nbd))).transpose(),
    np.vstack((np.ones(Nbd),np.linspace(1/Nbd,1,Nbd))).transpose()
),dtype=np.float32)
u_bd = np.hstack((
    -np.sin(np.pi*np.linspace(-1,1,Nbd)),
    np.zeros(2*Nbd)
),dtype=np.float32)
ν = 0.01

sampler = LatinHypercube(2)
xt = sampler.random(n=N)
xt[:,0] = 2*xt[:,0]-1

xt = np.vstack((xt_bd,xt),dtype=np.float32)

In [ ]:
def lossfn(y_true,y_pred):
    bd_loss = torch.sum(keras.losses.mean_squared_error(y_true,y_pred))/(3*Nbd)
    
    xt_tensor = torch.tensor(xt,requires_grad=True, device=y_pred.device)
    xt_tensor.grad = None
    u = model(xt_tensor).squeeze()
    xt_grad = torch.autograd.grad(u,xt_tensor,grad_outputs=torch.ones(u.shape,device=u.device),retain_graph=True,create_graph=True)[0]
    du_dx = xt_grad[:,0]
    du_dt = xt_grad[:,1]
    xt_grad2 = torch.autograd.grad(du_dx,xt_tensor,grad_outputs=torch.ones(u.shape,device=u.device),retain_graph=True)[0]
    d2u_dx2 = xt_grad2[:,0]

    residual = du_dt + u * du_dx - ( ν / np.pi) * d2u_dx2
    phys_loss = torch.sum(torch.pow(residual,2))/N
    
    return 5*bd_loss + phys_loss

In [ ]:
nnlayers = [20,20,20,20,20,20,20,20]

model = keras.Sequential([])
model.add(keras.Input(shape=(2,)))
for L in nnlayers:
    model.add(layers.Dense(L, activation='tanh'))
model.add(layers.Dense(1))

model.compile(loss=lossfn)

In [ ]:
# Unfortunately, this is where we must leave keras behind
# and write a torch-style training loop

def run_epoch(model, input, target):
    def closure():
        optimizer.zero_grad()
        output = model(input)
        loss = lossfn(target,output)
        loss.backward()
        return loss
    
    loss = optimizer.step(closure)

    return loss.item()

In [ ]:
epochs = 10000
patience = 10
threshold = 1e-4

losses = np.array([0.]*epochs)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
bar = tqdm(range(epochs))
for e in bar:
    model.train(True)
    loss = run_epoch(model,xt_bd,u_bd)
    losses[e] = loss
    bar.set_description(f'epoch {e+1}, loss: {loss:.3e}')
    
    if e > patience and np.max(np.abs(losses[e-patience:e]-loss))<threshold*loss:
        print('Model converged.')
        break
        

In [ ]:
x_full, t_full = np.meshgrid(np.linspace(-1,1,512),np.linspace(0,1,1024))
xt_full = np.vstack((x_full.ravel(),t_full.ravel())).transpose()
u_full = model.predict(xt_full,batch_size=10000)

u_full = np.reshape(u_full,(1024,512))

In [ ]:
plt.plot(losses[0:e])
plt.title('Training convergence');

In [ ]:
_, axs = plt.subplots(3)
axs[0].plot(x_full[-1,:],u_full[-1,:])
axs[1].imshow(u_full,origin='lower',extent=[-1, 1, 0, 1])
axs[2].plot(x_full[0,:],u_full[0,:])
plt.tight_layout()